In [1]:
import sys
sys.path.append('/opt/code/face/dual_shot')
import numpy as np
import collections
from prepare_data.generator import image_reader,layer_strides,map_size,e_scale,ratio
from prepare_data.model_target import init_anchors,iou_np,regress_target
from dual_conf import current_config as conf
from prepare_data.widerface import WIDERFaceDetection
from sklearn.utils import shuffle

Using TensorFlow backend.
/root/anaconda3/envs/keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/envs/keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## load gt

In [2]:
test_set = WIDERFaceDetection(conf.data_path, 'train', None, None)
image, gts, labels = image_reader(test_set)

Error bbox number to filter : 48,  bbox number: 159376


In [3]:
gts

array([[154., 230., 215., 287.],
       [352., 110., 405., 179.]])

## init anchors

In [4]:
anchors = init_anchors(layer_strides, map_size, ratio, e_scale)
anchors.shape

(34125, 4)

## target detail

In [5]:
iou_thread = 0.4
train_anchors = 10

In [6]:
# calculate iou matrix
iou = iou_np(gts, anchors)
# print('max_iou:', np.max(iou))
# get positive anchors which have iou bigger than 0.7 with any GT
iou_sign = np.where(iou > iou_thread, np.ones_like(iou), np.zeros_like(iou))
anchor_cls = np.argmax(iou_sign, axis=0)
# get positive anchors which has the highest iou with any GT,while the highest iou<0.7
row_max_ind = np.argmax(iou, axis=1)  # shape(2,)
# combine 2 types of positive anchors
anchor_cls = [anchor_cls[i] + 1 if i in row_max_ind else anchor_cls[i] for i in range(len(anchor_cls))]
anchor_cls = np.array(anchor_cls)
anchor_cls[anchor_cls > 1] = 1
# get non-positive index where iou with gt is <0.3
bad_ind = np.where(np.equal(anchor_cls, 0))[0]
bad_iou = iou[:, bad_ind]
bad_iou = np.max(bad_iou, axis=0)
bad_neg_ind = np.where(np.greater(0.3, bad_iou) * np.greater(bad_iou, 0))[0]
neg_ind = bad_ind[bad_neg_ind]

# randomly select positive anchors
pos_ind = np.where(np.not_equal(anchor_cls, 0))[0]
seed = np.random.randint(0, 1000, 1)[0]
pos_ind_chose = shuffle(pos_ind, random_state=seed)[0:train_anchors // 2]
neg_ind_chose = shuffle(neg_ind, random_state=seed)[0:train_anchors // 2]

In [7]:
neg_ind_chose

array([ 8057, 29534,  8069, 13805, 33865])

In [8]:
pos_ind_chose

array([32888, 32928, 32969, 32968, 32929])

In [9]:
np.sum(anchor_cls)

7

In [10]:
cls_target = np.full(shape=[conf.num_anchor],fill_value=0)
cls_target[pos_ind_chose]=1
cls_target[neg_ind_chose]=-1

In [11]:
cls_target.shape

(34125,)

In [12]:
col_max_ind = np.argmax(iou, axis=0)
col_max_ind.shape

(34125,)

In [13]:
gt_to_anchor = gts[col_max_ind]
reg_target = regress_target(anchors=anchors, gt_boxes=gt_to_anchor)

In [14]:
reg_target.shape

(34125, 4)

In [15]:
reg_target

array([[151.38298154, 297.97799897,   8.21473658,   9.49615337],
       [151.38298154, 201.27239404,   8.21473658,   7.57384039],
       [151.38298154, 190.21818784,   8.21473658,   7.3659755 ],
       ...,
       [ -1.92651587,  -4.1981001 ,  -9.81685269,  -9.00742783],
       [ -7.06511655,  -4.51316711,  -9.11394293,  -9.75483913],
       [ -7.06511655,  -8.97317536,  -9.11394293,  -7.83252615]])

## target func

In [16]:
def cal_target(gts=None, anchors=None, iou_thread=0.4, train_anchors=10):
    """ calculate target for 1 sample
    :param gts: 2-D array [num_gt,(y1,x1,y2,x2)]
    :param anchors: 2-D array, all initialized anchors [num_anchor,(y1,x1,y2,x2)]
    :param iou_thread: decimal,thread for positive anchor
    :param train_anchors: int, num of anchors to calculate loss
    :return:
    reg_target: 2-D array, [num_anchor,(dy,dx,dh,dw)]
    cls_target:2-D array, [num_anchor], 1,-1,0 for pos,neg and un-train anchors respectively
    """
    # calculate iou matrix
    iou = iou_np(gts, anchors)

    # get positive anchors which have iou bigger than 0.7 with any GT
    iou_sign = np.where(iou > iou_thread, np.ones_like(iou), np.zeros_like(iou))
    anchor_cls = np.argmax(iou_sign, axis=0)

    # get positive anchors which has the highest iou with any GT,while the highest iou<0.7
    row_max_ind = np.argmax(iou, axis=1)  # shape(2,)

    # combine 2 types of positive anchors
    anchor_cls = [anchor_cls[i] + 1 if i in row_max_ind else anchor_cls[i] for i in range(len(anchor_cls))]
    anchor_cls = np.array(anchor_cls)
    anchor_cls[anchor_cls > 1] = 1

    # get negative anchor index where iou with gt is between 0~0.3
    bad_ind = np.where(np.equal(anchor_cls, 0))[0]
    bad_iou = iou[:, bad_ind]
    bad_iou = np.max(bad_iou, axis=0)
    bad_neg_ind = np.where(np.greater(0.3, bad_iou) * np.greater(bad_iou, 0))[0]
    neg_ind = bad_ind[bad_neg_ind]

    # randomly select positive anchors
    pos_ind = np.where(np.not_equal(anchor_cls, 0))[0]
    seed = np.random.randint(0, 1000, 1)[0]
    pos_ind_chose = shuffle(pos_ind, random_state=seed)[0:train_anchors // 2]
    neg_ind_chose = shuffle(neg_ind, random_state=seed)[0:train_anchors // 2]

    # cls_target for all anchors
    cls_target = np.full(shape=[conf.num_anchor], fill_value=0)
    cls_target[pos_ind_chose] = 1
    cls_target[neg_ind_chose] = -1

    # reg_target for all anchors
    col_max_ind = np.argmax(iou, axis=0)
    gt_to_anchor = gts[col_max_ind]
    reg_target = regress_target(anchors=anchors, gt_boxes=gt_to_anchor)

    return reg_target, cls_target

In [17]:
reg_target, cls_target = cal_target(gts, anchors, iou_thread=0.4, train_anchors=10)

In [18]:
reg_target.shape

(34125, 4)

In [19]:
cls_target.shape

(34125,)

In [20]:
[cls_target[i] for i in range(34125) if cls_target[i]==1]

[1, 1, 1, 1, 1]

In [21]:
np.sum(cls_target)

0